In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [3]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0, 0]
  ng_refs = [2, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(10, 250), (600, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.9, 1.7), (1.4, 1.4, 1.4, 1.55, 1.6, 1.66)]
  wgt = [(0.7, 0.7), (0.7, 0.5, 0.5, 0.5, 0.5, 0.5)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.001999                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.002208
0.000750      2.0  0.002227                  2.0  0.002383
0.001052      3.0  0.002732                  3.0  0.002856
0.001476      4.0  0.003304                  4.0  0.003342
0.002070      5.0  0.003944                  5.0  0.003875
0.002904      6.0  0.004658                  6.0  0.004457
0.004074      7.0  0.005451                  7.0  0.005128
0.005714      8.0  0.006330                  8.0  0.005886
0.008015      9.0  0.007304                  9.0  0.006714
0.011243     10.0  0.008379                 10.0  0.007614
0.015771     11.0  0.009555                 11.0  0.008593
0.022122     12.0  0.010837                 12.0  0.009663
0.031031     13.0  0.012229                 13.0  0.010825
0.043528     14.0  0.013552                 14.0  0.011936
0.061057     15.0  0.014605                 15.0  0.012850
0.085645     16.0  0.015528                 16.0  0.013679
0.120136     17.0  0.016459                 17.0  0.014537
0.168516     18.0  0.017396                 18.0  0.015423
0.236378     19.0  0.018328                 19.0  0.016336
0.331549     20.0  0.019253                 20.0  0.017278
0.465100     21.0  0.020045                 21.0  0.018171
0.652400     22.0  0.018762                 22.0  0.017423
0.915100     23.0  0.015101                 23.0  0.014590
1.283650     24.0  0.011433                 24.0  0.011554
1.800600     25.0  0.008508                 25.0  0.008984
2.525700     26.0  0.006216                 26.0  0.006832
3.542800     27.0  0.004456                 27.0  0.005052
4.969550     28.0  0.003291                 28.0  0.003759
6.970850     29.0  0.002661                 29.0  0.002940
9.778100     30.0  0.002275                 30.0  0.002356
13.715850    31.0  0.001970                 31.0  0.001880
19.239350    32.0  0.001737                 32.0  0.001520
26.987250    33.0  0.001707                 33.0  0.001384
37.855300    34.0  0.001876                 34.0  0.001446
53.100050    35.0  0.002065                 35.0  0.001559
73.887500    36.0  0.002230                 36.0  0.001681
97.662500    37.0  0.002335                 37.0  0.001772
121.437500   38.0  0.002357                 38.0  0.001803
145.212500   39.0  0.003149                 39.0  0.002435
168.987500   40.0  0.003916                 40.0  0.003082
192.762500   41.0  0.004678                 41.0  0.003759
216.537500   42.0  0.005065                 42.0  0.004156
240.312500   43.0  0.005311                 43.0  0.004441
264.087500   44.0  0.005508                 44.0  0.004677
287.862500   45.0  0.005673                 45.0  0.004897
311.637500   46.0  0.006537                 46.0  0.005691
335.412500   47.0  0.008896                 47.0  0.007681
359.187500   48.0  0.012446                 48.0  0.010700
382.962500   49.0  0.020676                 49.0  0.017876
406.737500   50.0  0.033543                 50.0  0.029325
430.512500   51.0  0.048293                 51.0  0.042855
454.287500   52.0  0.063776                 52.0  0.057809
478.062500   53.0  0.079166                 53.0  0.073187
501.837500   54.0  0.094125                 54.0  0.087557
525.612500   55.0  0.108528                 55.0  0.100316
549.387500   56.0  0.122256                 56.0  0.112439
573.162500   57.0  0.135247                 57.0  0.125162
596.937500   58.0  0.147255                 58.0  0.138425
620.712500   59.0  0.158033                 59.0  0.151010
644.487500   60.0  0.167312                 60.0  0.161895
668.262500   61.0  0.174897                 61.0  0.170440
692.037500   62.0  0.180729                 62.0  0.176662
715.812500   63.0  0.184856                 63.0  0.181027
739.587500   64.0  0.187490                 64.0  0.184091
76

# Fluxes by Layer

CRD                              CLIRAD             \
                         flug          fldg      fnetg       flug       fldg   
pressure     level                                                             
0.000000e+00 1     -22.503850  0.000000e+00 -22.503850        NaN        NaN   
1.000000e-08 1            NaN           NaN        NaN -22.584901   0.000002   
6.244000e-04 2     -22.503850  2.669604e-07 -22.503850 -22.584901   0.000002   
8.759000e-04 3     -22.503850  3.798417e-07 -22.503850 -22.584901   0.000002   
1.228600e-03 4     -22.503850  5.544323e-07 -22.503850 -22.584901   0.000002   
1.723400e-03 5     -22.503851  8.246703e-07 -22.503850 -22.584901   0.000003   
2.417400e-03 6     -22.503851  1.242507e-06 -22.503849 -22.584901   0.000003   
3.390900e-03 7     -22.503851  1.888024e-06 -22.503849 -22.584901   0.000003   
4.756500e-03 8     -22.503851  2.884411e-06 -22.503848 -22.584901   0.000004   
6.672000e-03 9     -22.503851  4.420939e-06 -22.503847 -22.584901   0.000005   
9.358900e-03 10    -22.503851  6.788389e-06 -22.503844 -22.584901   0.000007   
1.312780e-02 11    -22.503851  1.043226e-05 -22.503840 -22.584900   0.000010   
1.841450e-02 12    -22.503851  1.603022e-05 -22.503834 -22.584900   0.000015   
2.583020e-02 13    -22.503850  2.461273e-05 -22.503825 -22.584898   0.000022   
3.623230e-02 14    -22.503848  3.774419e-05 -22.503810 -22.584895   0.000032   
5.082340e-02 15    -22.503844  5.758713e-05 -22.503786 -22.584890   0.000048   
7.129060e-02 16    -22.503838  8.695254e-05 -22.503751 -22.584883   0.000072   
1.000000e-01 17    -22.503828  1.299381e-04 -22.503698 -22.584871   0.000107   
1.402710e-01 18    -22.503812  1.926427e-04 -22.503620 -22.584853   0.000158   
1.967600e-01 19    -22.503787  2.837467e-04 -22.503503 -22.584825   0.000233   
2.759970e-01 20    -22.503747  4.154356e-04 -22.503331 -22.584782   0.000343   
3.871000e-01 21    -22.503682  6.045246e-04 -22.503078 -22.584716   0.000504   
5.431000e-01 22    -22.503580  8.729091e-04 -22.502707 -22.584614   0.000738   
7.617000e-01 23    -22.503441  1.218895e-03 -22.502222 -22.584476   0.001052   
1.068500e+00 24    -22.503284  1.611270e-03 -22.501673 -22.584320   0.001426   
1.498800e+00 25    -22.503124  2.033846e-03 -22.501090 -22.584158   0.001853   
2.102400e+00 26    -22.502966  2.484090e-03 -22.500482 -22.583995   0.002332   
2.949000e+00 27    -22.502819  2.960764e-03 -22.499858 -22.583842   0.002864   
4.136600e+00 28    -22.502698  3.466919e-03 -22.499231 -22.583712   0.003445   
5.802500e+00 29    -22.502613  4.030915e-03 -22.498582 -22.583616   0.004091   
8.139200e+00 30    -22.502563  4.717199e-03 -22.497846 -22.583554   0.004843   
1.141700e+01 31    -22.502562  5.600057e-03 -22.496962 -22.583543   0.005747   
1.601470e+01 32    -22.502654  6.764229e-03 -22.495889 -22.583624   0.006852   
2.246400e+01 33    -22.502901  8.338104e-03 -22.494563 -22.583854   0.008243   
3.151050e+01 34    -22.503342  1.060813e-02 -22.492733 -22.584265   0.010138   
4.420010e+01 35    -22.503978  1.406471e-02 -22.489914 -22.584862   0.012908   
6.200000e+01 36    -22.504860  1.930052e-02 -22.485560 -22.585710   0.017043   
8.577500e+01 37    -22.506018  2.673749e-02 -22.479280 -22.586868   0.022936   
1.095500e+02 38    -22.507181  3.447562e-02 -22.472705 -22.588067   0.029126   
1.333250e+02 39    -22.508445  4.237894e-02 -22.466066 -22.589378   0.035516   
1.571000e+02 40    -22.510405  5.320745e-02 -22.457197 -22.591379   0.044376   
1.808750e+02 41    -22.513272  6.710504e-02 -22.446167 -22.594258   0.055937   
2.046500e+02 42    -22.517351  8.435866e-02 -22.432992 -22.598288   0.070554   
2.284250e+02 43    -22.522665  1.039381e-01 -22.418726 -22.603476   0.087449   
2.522000e+02 44    -22.529440  1.256722e-01 -22.403768 -22.610002   0.106483   
2.759750e+02 45    -22.538152  1.498962e-01 -22.388255 -22.618240   0.127895   
2.997500e+02 46    -22.549366  1.770873e-01 -22.372278 -22.628585   0.152034   
3.235250e+02 47    -22.563

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')